In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Sprint16 公開されている実装を動かす
READMEを参考に上記実装を動かしてください。 
Faster R-CNN[1]の実装を動かします。

[1]Ren, S., He, K., Girshick, R., Sun, J.: Faster r-cnn: Towards real-time object detection with region proposal networks. In: Advances in neural information processing systems. (2015) 91–99

https://arxiv.org/pdf/1506.01497.pdf

以下のものを使用してください。Kerasを使用した実装です。

duckrabbits/ObjectDetection at master

## 【問題1】学習と推定
READMEを参考に上記実装を動かしてください。 



### (README)
Kerasを利用したFaster R-CNN実装により物体検出を行います。

データセットの用意
Kaggleからダウンロードしたthe-simpsons-characters-dataset.zipを解凍します。  
simpsons_dataset.zipを解凍します。  
解凍したsimpsons_datasetフォルダをプロジェクトフォルダ配下に格納します。  
annotation.txtもプロジェクトフォルダ配下に格納します。   
（kaggleからダウンロードしたannotation.txtではなく、ここにあるannotation.txtを使いましょう）

### 学習

以下のコマンドを実行してください。

python train.py -p annotation.txt

以下のように設定ファイルのパスが出力されるので記録してください。推定時に使います。

""" path to config file : ./save/train_20190309-220050_config.pickle """

エポック数なども引数で変更できます。詳細はtrain.pyを参照してください。


### 推定

以下のコマンドを実行してください。

python predict.py -i [推定したい画像を入れたディレクトリのパス] -c [学習時に保存された設定ファイルのパス]


### jupyter環境で実施

In [0]:
#kaggle　のシンプソンズデータセットをインストール
# !pip install kaggle

In [0]:
# ls -a

In [0]:
# https://www.kaggle.com/　へアクセスし、ログイン後に右上のプロフィール画像をクリック、さらに"My Account"をクリックする
# このサイトの「API」項目にある"Create New API Token"をクリックすると、kaggle.jsonファイルが自動的にダウンロードされる
# ローカルで、kaggle.json（ダウンロードフォルダにあるはず）をエディターで開く。
# このセルの以下のコードにある token = {'username':'***','key':'***'} における「***」部分を、
# ダウンロードしたkaggle.jsonを参照して書き換え、このセルを実行する

# import json

# token = {'username':'kaorisugi','key':'45dfd98b8e531a606d8922234e841c6d'}
# with open('/content/.kaggle/kaggle.json', 'w') as file:
#     json.dump(token, file)

In [0]:
# !chmod 600 /content/.kaggle/kaggle.json
# !cp /content/.kaggle/kaggle.json /root/.kaggle/kaggle.json

In [0]:
# simpsonデータセットのAPIcommandを叩いてデータセットをダウンロードする
# APIcommandはこちらのサイトに。https://www.kaggle.com/alexattia/the-simpsons-characters-dataset

# !kaggle datasets download -d alexattia/the-simpsons-characters-dataset

# !unzip /content/the-simpsons-characters-dataset.zip  -d Simpsons
# # train用のデータセットを解凍

# !unzip /content/Simpsons/simpsons_dataset.zip

In [0]:
# !git clone https://github.com/duckrabbits/ObjectDetection.git

In [2]:
# import os

# os.chdir('/content/ObjectDetection')
# print(os.getcwd())


In [4]:
pwd

'/Users/user/programing/diveintocode-ml/Sprint16'

In [11]:
ls

README.md        model/           save/            train.py
annotation.txt   predict.py       simpson_testset/


In [6]:
cd ObjectDetection/ 

/Users/user/programing/diveintocode-ml/Sprint16/ObjectDetection


In [14]:
# ! python train.py -p annotation.txt

In [0]:
# 修正箇所
# tf ==> channels_last
# K.image_dim_ordering() ==> K.image_data_format
# gitのanotation.txtとkaggle_dataのファイル 名を合わせた
# その他ファイル の移動など

In [14]:
# ! python train.py -p annotation.txt#実行

### 【問題2】コードリーディング
上記実装のコードリーディングを行ってください。

### RPNを実現しているコードはどこか
論文によると、今までのFast_rcnnなどはResion Proposalの計算時間が課題だった。  
今回FasterではRPNを採用することで高速化している。（一番の売り）
### RoIプーリングを実現しているコードはどこか
物体検出の際、RPNから出力されるFeature map を　その後の分類作業に渡し、情報共有することで高速化を実現している　。  
情報共有の際にデータの形状を合わせる必要があり、それを実施しているのがRoIプーリング（これも売りの一つ）

In [0]:
from __future__ import division

import os
import sys
import time
import argparse
from datetime import datetime
import numpy as np
import pickle

"""Faster_rcnnのモデル"""
from model import faster_rcnn

from model import config, data_generators
from model.parser import get_data

"""ROIプーリングのモデル"""
import model.roi_helpers as roi_helpers

from keras import backend as K
from keras.utils import generic_utils

sys.setrecursionlimit(10000)

parser = argparse.ArgumentParser(formatter_class=argparse.RawTextHelpFormatter, description='Object Detection')
parser.add_argument("-p", "--path", default=None, help="path to annotation file")
parser.add_argument("--save_dir", default="./save", help="path to save directory")
parser.add_argument('--n_epochs', default=10, type=int, metavar='N',
                    help='number of epochs')
parser.add_argument('--n_iters', default=100, type=int, metavar='N',
                    help='number of iterations')
parser.add_argument('--horizontal_flips', action='store_true',
                    help='augument with horizontal flips (Default:False)')
parser.add_argument('--vertical_flips', action='store_true',
                    help='augument with horizontal flips (Default:False)')
parser.add_argument('--rot_90', action='store_true',
                    help='augument with 90 degree rotations (Default:False)')

def main():
    args = parser.parse_args()
    time_stamp = "{0:%Y%m%d-%H%M%S}".format(datetime.now())
    save_name = os.path.join(args.save_dir, "train_{}".format(time_stamp))

    if not(os.path.isdir(args.save_dir)):
        os.makedirs(args.save_dir)
    if args.path == None:
        raise OSError("path to annotation file must be required.")
    C = config.Config()
    C.config_filename = save_name + "_config.pickle"
    C.model_path = save_name + "_model.hdf5"
    C.use_horizontal_flips = bool(args.horizontal_flips)
    C.use_vertical_flips = bool(args.vertical_flips)
    C.rot_90 = bool(args.rot_90)
    all_imgs, classes_count, class_mapping = get_data(args.path)
    C.class_mapping = class_mapping

    with open(C.config_filename, 'wb') as config_f:
        pickle.dump(C,config_f)
        print("-------------------------------")
        print('path to config file : {}'.format(C.config_filename))
        print("-------------------------------")

    train_imgs = [s for s in all_imgs if s['imageset'] == 'trainval']
    val_imgs = [s for s in all_imgs if s['imageset'] == 'test']

    data_gen_train = data_generators.get_anchor_gt(train_imgs, classes_count, C, K.image_dim_ordering(), mode='train')
    data_gen_val = data_generators.get_anchor_gt(val_imgs, classes_count, C, K.image_dim_ordering(), mode='val')
    
    """fasterのモデルからrpnとclassifier等のモデルをもらっている"""
    model_rpn, model_classifier, model_all = faster_rcnn.get_model(C, classes_count)
    
    losses = np.zeros((args.n_iters, 5))
    rpn_accuracy_rpn_monitor, rpn_accuracy_for_epoch = [], []

    best_loss = np.Inf

    with open('out.csv', 'w') as f:
        f.write('Accuracy,RPN classifier,RPN regression,Detector classifier,Detector regression,Total')
        f.write('\t')

    iter_num = 0

    t0 = start_time = time.time()
    try:
        for epoch_num in range(args.n_epochs):
            progbar = generic_utils.Progbar(args.n_iters)
            print('Epoch {}/{}'.format(epoch_num + 1, args.n_epochs))

            while True:
                try:
                    if len(rpn_accuracy_rpn_monitor) == args.n_iters and C.verbose:
                        mean_overlapping_bboxes = float(sum(rpn_accuracy_rpn_monitor))/len(rpn_accuracy_rpn_monitor)
                        rpn_accuracy_rpn_monitor = []
                        print('Average number of overlapping bounding boxes from RPN = {} for {} previous iterations'.format(mean_overlapping_bboxes, args.n_iters))
                        if mean_overlapping_bboxes == 0:
                            print('RPN is not producing bounding boxes that overlap the ground truth boxes. Check RPN settings or keep training.')
                    X, Y, img_data = next(data_gen_train)
                    
                    """Resion Proposalの学習、推定、ROIプーリングをしている。"""
                    loss_rpn = model_rpn.train_on_batch(X, Y)
                    P_rpn = model_rpn.predict_on_batch(X)
                    R = roi_helpers.rpn_to_roi(P_rpn[0], P_rpn[1], C, K.image_dim_ordering(), use_regr=True, overlap_thresh=0.7, max_boxes=300)
                    
                    # note: calc_iou converts from (x1,y1,x2,y2) to (x,y,w,h) format
                    X2, Y1, Y2 = roi_helpers.calc_iou(R, img_data, C, class_mapping)
                    


                    neg_samples = np.where(Y1[0, :, -1] == 1)
                    pos_samples = np.where(Y1[0, :, -1] == 0)
                    if len(neg_samples) > 0:
                        neg_samples = neg_samples[0]
                    else:
                        neg_samples = []

                    if len(pos_samples) > 0:
                        pos_samples = pos_samples[0]
                    else:
                        pos_samples = []

                    rpn_accuracy_rpn_monitor.append(len(pos_samples))
                    rpn_accuracy_for_epoch.append((len(pos_samples)))
                    if len(pos_samples) < C.num_rois//2:
                        selected_pos_samples = pos_samples.tolist()
                    else:
                        selected_pos_samples = np.random.choice(pos_samples, C.num_rois//2, replace=False).tolist()
                    try:
                        selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=False).tolist()
                    except:
                        selected_neg_samples = np.random.choice(neg_samples, C.num_rois - len(selected_pos_samples), replace=True).tolist()

                    sel_samples = selected_pos_samples + selected_neg_samples

                    loss_class = model_classifier.train_on_batch([X, X2[:, sel_samples, :]], [Y1[:, sel_samples, :], Y2[:, sel_samples, :]])

                    if iter_num == args.n_iters:
                        loss_rpn_cls = np.mean(losses[:, 0])
                        loss_rpn_regr = np.mean(losses[:, 1])
                        loss_class_cls = np.mean(losses[:, 2])
                        loss_class_regr = np.mean(losses[:, 3])
                        class_acc = np.mean(losses[:, 4])

                        mean_overlapping_bboxes = float(sum(rpn_accuracy_for_epoch)) / len(rpn_accuracy_for_epoch)
                        rpn_accuracy_for_epoch = []

                        if C.verbose:
                            print('Mean number of bounding boxes from RPN overlapping ground truth boxes: {}'.format(mean_overlapping_bboxes))
                            print('Classifier accuracy for bounding boxes from RPN: {}'.format(class_acc))
                            print('Loss RPN classifier: {}'.format(loss_rpn_cls))
                            print('Loss RPN regression: {}'.format(loss_rpn_regr))
                            print('Loss Detector classifier: {}'.format(loss_class_cls))
                            print('Loss Detector regression: {}'.format(loss_class_regr))
                            print('Elapsed time: {}[s]'.format(time.time() - start_time))

                        target_text_file = open('out.csv', 'a')
                        target_text_file.write('{},{},{},{},{},{}'.format(class_acc, loss_rpn_cls,
                                                loss_rpn_regr, loss_class_cls, loss_class_regr,
                                                loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr))
                        target_text_file.write('\t')

                        curr_loss = loss_rpn_cls + loss_rpn_regr + loss_class_cls + loss_class_regr
                        iter_num = 0
                        start_time = time.time()

                        if curr_loss < best_loss:
                            if C.verbose:
                                print('Total loss decreased from {} to {}, saving weights'.format(best_loss,curr_loss))
                            best_loss = curr_loss
                            model_all.save_weights(C.model_path)
                        break

                    losses[iter_num, 0] = loss_rpn[1]
                    losses[iter_num, 1] = loss_rpn[2]
                    losses[iter_num, 2] = loss_class[1]
                    losses[iter_num, 3] = loss_class[2]
                    losses[iter_num, 4] = loss_class[3]
                    iter_num += 1

                    progbar.update(iter_num, [('rpn_cls', np.mean(losses[:iter_num, 0])), ('rpn_regr', np.mean(losses[:iter_num, 1])),
                                              ('detector_cls', np.mean(losses[:iter_num, 2])), ('detector_regr', np.mean(losses[:iter_num, 3]))])

                except Exception as e:
                    print('Exception: {}'.format(e))
                    continue

    except KeyboardInterrupt:
        t1 = time.time()
        print('\nIt took {:.2f}s'.format(t1-t0))
        sys.exit('Keyboard Interrupt')

    print("training is done")
    print("-------------------------------")
    print('path to config file : {}'.format(C.config_filename))
    print("-------------------------------")

if __name__ == '__main__':
    main()

### 【問題3】学習済みの重みによる推定
学習済みの重みを使い推定を行う方法がREADME.mdのQuick Startの項に記載されています。  
まずはこの通りにして各自何かしらの画像や動画に対して検出を行ってください。  
出力結果を課題の一部として提出してください。

# 3.YOLOv3
シンプソンズのデータセットをFaster R-CNN以外の手法で学習・推定を行います。YOLOv3[2]のKeras実装を使います。

qqwweee/keras-yolo3: A Keras implementation of YOLOv3 (Tensorflow backend)

[2]Jeseph Redmon, Ali Farhadi. YOLOv3: An Incremental Improvement

https://pjreddie.com/media/files/papers/YOLOv3.pdf  
https://github.com/qqwweee/keras-yolo3

In [15]:
pwd

'/Users/user/programing/diveintocode-ml/Sprint16/ObjectDetection'

In [8]:
# !git clone https://github.com/qqwweee/keras-yolo3.git

In [17]:
cd keras-yolo3

/Users/user/programing/diveintocode-ml/Sprint16/ObjectDetection/keras-yolo3


In [9]:
# !brew install wget

In [10]:
# !wget https://pjreddie.com/media/files/yolov3.weights

In [11]:
#事前学習済みの重みをロード
# !python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5

In [12]:
# !python train.py

In [13]:
# !python yolo_video.py --image 

## 【問題4】学習のためのファイルを作成
新しいデータ（シンプソンズデータセット）を学習します。README.mdのTrainingの項を読み、シンプソンズデータセットを学習するために必要なファイルを作成してください。

アノテーションファイルの形式が問題1の実装とは異なるため変換する必要があります。
《アノテーションファイルの形式》
image_file_path,xmin,ymin,xmax,ymax,label

In [5]:
#simpsonの画像データをYOLOへ移動
# !mv simpson_dataset keras-yolo3/

In [7]:
# !python convert.py yolov3.cfg yolov3.weights model_data/yolo.h5